## Plan
1. use 2d unet from [segmentation_models.pytorch](https://github.com/qubvel/segmentation_models.pytorch) repo
2. add cv with n folds (let's start with n = 5)
3. train on:
  - resized image
  - image fragments


In [1]:
import torch
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm
import cv2

In [2]:
#!pip install --upgrade segmentation-models-pytorch
SEED = 2768
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
SIZE=320#224

In [3]:
import segmentation_models_pytorch as smp


In [4]:
with np.load('../data/data_train.npz') as dataset:
        train_dataset = dataset['data']
        
with np.load('../data/data_test_1.npz') as dataset:
        test_dataset = dataset['data']

with np.load('../data/labels_train.npz') as labels:
        train_labels = labels['labels']

In [5]:
def fast_glcm(img, vmin=0, vmax=255, nbit=8, kernel_size=5):
    mi, ma = vmin, vmax
    ks = kernel_size
    h,w = img.shape

    # digitize
    bins = np.linspace(mi, ma+1, nbit+1)
    gl1 = np.digitize(img, bins) - 1
    gl2 = np.append(gl1[:,1:], gl1[:,-1:], axis=1)

    # make glcm
    glcm = np.zeros((nbit, nbit, h, w), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            mask = ((gl1==i) & (gl2==j))
            glcm[i,j, mask] = 1

    kernel = np.ones((ks, ks), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            glcm[i,j] = cv2.filter2D(glcm[i,j], -1, kernel)

    glcm = glcm.astype(np.float32)
    return glcm

def fast_glcm_max(img, vmin=0, vmax=255, nbit=2, ks=5):
    '''
    calc glcm max
    '''
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    max_  = np.max(glcm, axis=(0,1))
    return max_

# Preprocessing our image, inclusing  applying fast_glcm_max function, resizing and normalizing pixel values 
def process_img_label(img, label=None, size=224):

    img = fast_glcm_max(img)

    img = np.expand_dims(img, axis=2)#.astype('float32')


    img = cv2.resize(img, (size, size))


    img = cv2.normalize(img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX).astype(np.uint8)

    try:
        label = np.expand_dims(label, axis=2).astype(np.uint8)
        label = cv2.resize(label, (size, size))

        return img, label

    except:
        return img

In [6]:
test_dataset.shape

(1006, 782, 251)

In [7]:
training_img_data = []
test_img_data = []
training_label_data = []

for i in tqdm(range(0, 590)):
    img = train_dataset[:, :, i]
    label = train_labels[:, :, i]

    img, label = process_img_label(img, label, size=SIZE)

    training_img_data.append(img) 
    training_label_data.append(label)
for i in tqdm(range(0, 251)):
    img = test_dataset[:, :, i]
    img = process_img_label(img, size=SIZE)
    test_img_data.append(img)

/home/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/ipykernel_launcher.py:48: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [8]:
training_img_data = np.asarray(training_img_data)
test_img_data = np.asarray(test_img_data)
training_label_data = np.asarray(training_label_data)
training_img_data = np.expand_dims(training_img_data, 1)
test_img_data = np.expand_dims(test_img_data, 1)
training_img_data.shape, training_label_data.shape, test_img_data.shape

((590, 1, 320, 320), (590, 320, 320), (251, 1, 320, 320))

In [9]:
import solt
import solt.transforms as slt

stream = solt.Stream([
    slt.Rotate(angle_range=(0, 360), p=1, padding='r'),
     slt.Shear(range_x=0.3, range_y=0.8, p=0.5, padding='r'),
     slt.Flip(axis=1, p=0.5),
    slt.Flip(axis=0, p=0.5),
    #  slt.IntensityRemap(),
    slt.Brightness(brightness_range=(0, 1)),
     slt.Pad(200),
     slt.Scale(range_x=(0.8, 1.3), padding='r', range_y=(0.8, 1.3), same=False, p=0.5),
     slt.Contrast(p=0.5),
    #   solt.SelectiveStream([
    #     slt.CutOut(40, p=1),
    #     slt.CutOut(50, p=1),
    #     slt.CutOut(10, p=1),
    #     solt.Stream(),
    #     solt.Stream(),
    # ], n=3),
    # slt.Crop((224, 224)),
    solt.SelectiveStream([
        slt.GammaCorrection(gamma_range=0.5, p=1),
        slt.Noise(gain_range=0.1, p=1),
        slt.Blur()    
    ], n=3)
])

In [10]:
ENCODER = 'timm-efficientnet-b4'#'resnet34' #'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'

ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)



In [11]:
#model(torch.randn(10, 1, 224, 224)).shape

In [12]:
training_img_data.shape

(590, 1, 320, 320)

In [13]:
from sklearn.model_selection import KFold

In [14]:
import albumentations as albu

def get_training_augmentation(size=224):
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=size, min_width=size, always_apply=True, border_mode=0),
        albu.RandomCrop(height=size, width=size, always_apply=True),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation(do_pad=False):
    """Add paddings to make image shape divisible by 32"""
    if do_pad:
        test_transform = [
        albu.PadIfNeeded(384, 480)
        ]
    else:
        test_transform = []
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [15]:
from torch.utils.data import Dataset, DataLoader

#img_aug, mask_aug = stream({'image': train_data[:, 0], 'mask':train_labels[:,0]}, return_torch=False, ).data
class SeismicDataset(Dataset):
    def __init__(self, data, mask_data=None, transform=None, augmentation=None, preprocessing=None, labels=[]):
        self.data = data
        self.mask_data = mask_data
        self.transform = transform
        self.augmentation=augmentation
        self.preprocessing = preprocessing
        self.labels=labels
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        img = self.data[idx, 0]
        img_new = np.zeros(img.shape + (3,), dtype=img.dtype)
        for i in range(3):
            img_new[:, :, i] = img[:, :]
        img = img_new
        #print(img.shape, img.dtype)
        if self.mask_data is None:
            if self.transform is not None:
                img = self.transform(img)
            if self.augmentation:
                sample = self.augmentation(image=img)
                img = sample['image']

            # apply preprocessing
            if self.preprocessing:
                sample = self.preprocessing(image=img)
                img = sample['image']
            return img
        mask = self.mask_data[idx]
        masks = np.zeros(mask.shape+(len(self.labels), ), dtype=np.float)
        for i, label in enumerate(self.labels):
            masks[:, :, i] = mask[:, :] == label
            #print(np.sum(mask==label))
        #if self.transform is not None:
        #    img, label = self.transform(img, label)
        if self.augmentation:
            sample = self.augmentation(image=img, mask=masks)
            img, masks = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=img, mask=masks)
            img, masks = sample['image'], sample['mask']
            
        return img, masks

In [16]:
# train model for 40 epochs
def do_train(model, train_epoch, valid_epoch, train_loader, val_loader, n_epochs=40, fold=0):
    max_score = 0

    for i in range(0, n_epochs):

        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(val_loader)

        # do something (save model, change lr, etc.)
        if max_score < valid_logs['iou_score']:
            max_score = valid_logs['iou_score']
            torch.save(model, f'./solution2/best_model_{fold}.pth')
            print('Model saved!')

        if i == 25:
            optimizer.param_groups[0]['lr'] = 1e-5
            print('Decrease decoder learning rate to 1e-5!')

In [17]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

ACTIVATION = 'softmax2d' # could be None for logits or 'softmax2d' for multicalss segmentation
NFOLDS=5
labels = np.unique(training_label_data)

In [18]:

kf = KFold(random_state=SEED, n_splits=NFOLDS, shuffle=True)
for fold, (train_ids, val_ids) in enumerate(kf.split(training_img_data)):
    train_data = training_img_data[train_ids]
    train_labels = training_label_data[train_ids]
    val_data = training_img_data[val_ids]
    val_labels = training_label_data[val_ids]
    
    train_ds = SeismicDataset(
        train_data, train_labels,
        preprocessing=get_preprocessing(preprocessing_fn),
        augmentation=get_training_augmentation(SIZE),
        labels=labels
    )
    val_ds = SeismicDataset(
        val_data, val_labels,
        preprocessing=get_preprocessing(preprocessing_fn),
        augmentation=get_validation_augmentation(),
        labels=labels
    )
    
    train_loader = DataLoader(train_ds, batch_size=8)
    val_loader = DataLoader(val_ds, batch_size=4)
    model = smp.Unet(
        ENCODER, classes=len(labels), in_channels=3,
        encoder_weights=ENCODER_WEIGHTS,
        activation=ACTIVATION,
    )
    
    optimizer = torch.optim.Adam([ 
        dict(params=model.parameters(), lr=0.0001),
    ])

    train_epoch = smp.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )

    do_train(model, train_epoch, valid_epoch, train_loader, val_loader, fold=fold)
    del model
    #break


Epoch: 0
valid: 100%|██████████| 30/30 [00:01<00:00, 25.67it/s, dice_loss - 0.6155, iou_score - 0.2553]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:01<00:00, 24.71it/s, dice_loss - 0.4191, iou_score - 0.5215]
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:01<00:00, 24.66it/s, dice_loss - 0.3746, iou_score - 0.5474]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:01<00:00, 24.64it/s, dice_loss - 0.3395, iou_score - 0.5699]
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:01<00:00, 23.47it/s, dice_loss - 0.2928, iou_score - 0.5946]
Model saved!

Epoch: 5
valid: 100%|██████████| 30/30 [00:01<00:00, 24.84it/s, dice_loss - 0.2362, iou_score - 0.7126]
Model saved!

Epoch: 6
valid: 100%|██████████| 30/30 [00:01<00:00, 25.31it/s, dice_loss - 0.2084, iou_score - 0.7241]
Model saved!

Epoch: 7
valid: 100%|██████████| 30/30 [00:01<00:00, 25.26it/s, dice_loss - 0.1982, iou_score - 0.7308]
Model saved!

Epoch: 8
valid: 100%|██████████| 30/30 [00:01<00:00, 25

valid: 100%|██████████| 30/30 [00:01<00:00, 24.96it/s, dice_loss - 0.06696, iou_score - 0.8834]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:01<00:00, 25.39it/s, dice_loss - 0.6519, iou_score - 0.2216]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:01<00:00, 24.83it/s, dice_loss - 0.3201, iou_score - 0.6094]
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:01<00:00, 25.28it/s, dice_loss - 0.2696, iou_score - 0.6575]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:01<00:00, 21.42it/s, dice_loss - 0.231, iou_score - 0.6952] 
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:01<00:00, 26.22it/s, dice_loss - 0.2103, iou_score - 0.7072]
Model saved!

Epoch: 5
valid: 100%|██████████| 30/30 [00:01<00:00, 23.77it/s, dice_loss - 0.2042, iou_score - 0.7138]
Model saved!

Epoch: 6
valid: 100%|██████████| 30/30 [00:01<00:00, 25.03it/s, dice_loss - 0.1944, iou_score - 0.7241]
Model saved!

Epoch: 7
valid: 100%|██████████| 30/30 [00:01<00:00, 24.42it/s, 

valid: 100%|██████████| 30/30 [00:01<00:00, 26.59it/s, dice_loss - 0.09528, iou_score - 0.8352]

Epoch: 38
valid: 100%|██████████| 30/30 [00:01<00:00, 26.18it/s, dice_loss - 0.09417, iou_score - 0.8373]

Epoch: 39
valid: 100%|██████████| 30/30 [00:01<00:00, 26.36it/s, dice_loss - 0.09356, iou_score - 0.8393]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:01<00:00, 26.24it/s, dice_loss - 0.4702, iou_score - 0.4553]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:01<00:00, 26.66it/s, dice_loss - 0.3762, iou_score - 0.5268]
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:01<00:00, 25.42it/s, dice_loss - 0.3075, iou_score - 0.5557]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:01<00:00, 23.23it/s, dice_loss - 0.2545, iou_score - 0.6321]
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:01<00:00, 23.82it/s, dice_loss - 0.1708, iou_score - 0.7847]
Model saved!

Epoch: 5
valid: 100%|██████████| 30/30 [00:01<00:00, 23.85it/s, dice_loss - 0.1533, io

valid: 100%|██████████| 30/30 [00:01<00:00, 25.87it/s, dice_loss - 0.0847, iou_score - 0.8531] 

Epoch: 37
valid: 100%|██████████| 30/30 [00:01<00:00, 25.05it/s, dice_loss - 0.08415, iou_score - 0.8556]
Model saved!

Epoch: 38
valid: 100%|██████████| 30/30 [00:01<00:00, 25.01it/s, dice_loss - 0.08401, iou_score - 0.8544]

Epoch: 39
valid: 100%|██████████| 30/30 [00:01<00:00, 24.01it/s, dice_loss - 0.08362, iou_score - 0.8559]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:01<00:00, 23.59it/s, dice_loss - 0.7115, iou_score - 0.1455]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:01<00:00, 24.10it/s, dice_loss - 0.4203, iou_score - 0.5264]
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:01<00:00, 25.30it/s, dice_loss - 0.3408, iou_score - 0.5887]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:01<00:00, 26.07it/s, dice_loss - 0.2858, iou_score - 0.6164]
Model saved!

Epoch: 4
valid: 100%|██████████| 30/30 [00:01<00:00, 25.26it/s, dice_loss - 0.2048, 

valid: 100%|██████████| 30/30 [00:01<00:00, 26.34it/s, dice_loss - 0.08545, iou_score - 0.846] 
Model saved!

Epoch: 36
valid: 100%|██████████| 30/30 [00:01<00:00, 23.58it/s, dice_loss - 0.08637, iou_score - 0.8434]

Epoch: 37
valid: 100%|██████████| 30/30 [00:01<00:00, 24.95it/s, dice_loss - 0.08467, iou_score - 0.8473]
Model saved!

Epoch: 38
valid: 100%|██████████| 30/30 [00:01<00:00, 24.96it/s, dice_loss - 0.0859, iou_score - 0.8469] 

Epoch: 39
valid: 100%|██████████| 30/30 [00:01<00:00, 25.57it/s, dice_loss - 0.08584, iou_score - 0.8496]
Model saved!

Epoch: 0
valid: 100%|██████████| 30/30 [00:01<00:00, 26.01it/s, dice_loss - 0.6863, iou_score - 0.1733]
Model saved!

Epoch: 1
valid: 100%|██████████| 30/30 [00:01<00:00, 24.85it/s, dice_loss - 0.4312, iou_score - 0.525] 
Model saved!

Epoch: 2
valid: 100%|██████████| 30/30 [00:01<00:00, 24.83it/s, dice_loss - 0.3148, iou_score - 0.5908]
Model saved!

Epoch: 3
valid: 100%|██████████| 30/30 [00:01<00:00, 22.35it/s, dice_loss - 0.2685

valid: 100%|██████████| 30/30 [00:01<00:00, 26.44it/s, dice_loss - 0.08606, iou_score - 0.8547]
Model saved!

Epoch: 35
valid: 100%|██████████| 30/30 [00:01<00:00, 22.10it/s, dice_loss - 0.08764, iou_score - 0.8511]

Epoch: 36
valid: 100%|██████████| 30/30 [00:01<00:00, 25.10it/s, dice_loss - 0.08643, iou_score - 0.854] 

Epoch: 37
valid: 100%|██████████| 30/30 [00:01<00:00, 24.89it/s, dice_loss - 0.08585, iou_score - 0.8541]

Epoch: 38
valid: 100%|██████████| 30/30 [00:01<00:00, 25.07it/s, dice_loss - 0.08604, iou_score - 0.8544]

Epoch: 39
valid: 100%|██████████| 30/30 [00:01<00:00, 25.33it/s, dice_loss - 0.08636, iou_score - 0.8529]


In [19]:
#out = model(torch.from_numpy(train_data).float())

In [20]:
training_img_data.shape

(590, 1, 320, 320)

In [21]:
test_img_data.shape

(251, 1, 320, 320)

In [22]:
test_ds = SeismicDataset(
        test_img_data,
        preprocessing=get_preprocessing(preprocessing_fn),
        augmentation=get_validation_augmentation(),
        labels=labels
)
val_loader = DataLoader(test_ds, batch_size=4)

In [23]:
with torch.no_grad():
    all_predictions = []
    for fold in np.arange(NFOLDS):
        fold_predictions = []
        model = torch.load(f'./solution2/best_model_{fold}.pth')
        model = model.to(DEVICE)
        for batch in val_loader:
            predictions = model(batch.to(DEVICE))
            predictions = predictions.detach().cpu().numpy()
            fold_predictions.append(predictions)
        fold_predictions = np.concatenate(fold_predictions, 0)
        all_predictions.append(fold_predictions)
        del model
    #torch.load()
    #for batch in val_loader:

In [24]:
predictions = np.mean(all_predictions, 0)

In [29]:
all_predictions = []
for pred in predictions:
    p = cv2.resize(np.moveaxis(pred, 0, -1), (782, 1006))
    p = np.argmax(p, -1)
    all_predictions.append(labels[p])
all_predictions = np.stack(all_predictions, -1).astype(np.int)

In [30]:
with np.load('../data/sample_submission_1.npz') as dataset:
    print(list(dataset.keys()))
    sample = dataset['prediction']

['prediction']


In [31]:
assert sample.shape==all_predictions.shape

In [32]:
np.savez_compressed("solution2/submission_solution2.npz", prediction=all_predictions)